DATA PREPARATION

In [2]:
#I decided to use beautiful soup library to gather data from Wikipedia

# Beautifulsoup installation 
!conda install -c anaconda beautifulsoup4 --yes

print('done')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    numpy-base-1.15.4          |   py36h81de0dd_0         4.2 MB  anaconda
    beautifulsoup4-4.8.2       |           py36_0         161 KB  anaconda
    numpy-1.15.4               |   py36h1d66e8a_0          35 KB  anaconda
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    soupsieve-1.9.5            |           py36_0          61 KB  anaconda
    mkl_fft-1.0.6              |   py36h7dd41cf_0         150 KB  anaconda
    certifi-2019.11.28         |           py36_0         156 KB  an

In [3]:
#updating
#conda update -n base -c deafaults conda 
#print('update complete')

In [4]:
#lxml install
!conda install lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.3.0                 |   py36hefd8a0e_0         1.5 MB
    certifi-2019.11.28         |           py36_0         156 KB
    openssl-1.1.1d             |       h7b6447c_3         3.7 MB
    ------------------------------------------------------------
                                           Total:         5.4 MB

The following packages will be UPDATED:

    certifi: 2019.11.28-py36_0    anaconda --> 2019.11.28-py36_0   
    lxml:    4.2.5-py37hefd8a0e_0          --> 4.3.0-py36hefd8a0e_0

The following packages w

In [5]:
# html5lib install
!conda install html5lib

Solving environment: \ 
  - conda-forge::html5lib-0.9999999-py36_0
  - defaults::html5lib-0.9999999-py36done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [6]:
# Requests installation to open URL
!conda install requests --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - requests


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    requests-2.22.0            |           py36_1          90 KB

The following packages will be UPDATED:

    requests: 2.22.0-py36_1 conda-forge --> 2.22.0-py36_1


requests-2.22.0      | 90 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [7]:
#Importing library
from bs4 import BeautifulSoup
import requests
print('library imported')
print('\n\n')


library imported





DATA PREPARATION USING PANDA

In [8]:
import pandas as pd
import numpy as np

In [9]:
# Read table from wiki page
data_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)

# to show the table and verify if it is correct
table = data_list[0]

# creating a dataframe
data_df = pd.DataFrame(table)

First operation

In [14]:
#Cleaning the table by ignoring cells with a borough that is Not assigned.
filter1 = data_df.drop(data_df[data_df['Borough'] == 'Not assigned'].index, axis=0)
print('shape after first filter:', filter1.shape)

Second Operation

In [14]:
#Clean data by assigning the same borough when a neighbourhood is not assigned
#Looking for indexes where neighbourhoods are not assigned
na_index = filter1.index[filter1['Neighbourhood'] == 'Not assigned'].tolist()

# Considering that we have only one single not assigned value, I assign Borough name to Neighbourhood by hand
filter1['Neighbourhood'] = filter1['Neighbourhood'].replace('Not assigned', 'Queen\'s Park')

Third Operation

In [60]:
#Combining rows with the same postal code and different neighbourhood by using function groupby
    # grouped by Postcode, then it will be renamed in Postalcode
    # Neighbourhoods belonging to the same postal code will be added together separated by a comma using .agg and.join function
    # Index is reset
    
groupby1 =filter1.groupby('Postcode').agg({'Borough' : 'first', 'Neighbourhood' : ','.join}).reset_index().reindex(columns=filter1.columns)

print('After groupby function we have a dataframe:\n', type(groupby1))

# Column name assignment
groupby1.columns = ['PostalCode', 'Borough','Neighborhood']

After groupby function we have a dataframe:
 <class 'pandas.core.frame.DataFrame'>


Final dimension check --> Shape function

In [62]:
# Checking dataframe dimensions
print('\n\nfinal dataframe dimensions:', groupby1.shape)

#Printing dataframe
groupby1.head(10)



final dataframe dimensions: (103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


PART 2

In [98]:
# Importing Geospatial Data
lat_long = pd.read_csv('http://cocl.us/Geospatial_data')
lat_long.head(10)

In [110]:
#Checking geospatial dimensions
lat_long.shape # Dimensions are the same as the final dataframe with postalcodes

(103, 3)

In [112]:
#sorting previous dataframe in ascending order with respect to Postalcode column
groupby1.sort_values(by=['PostalCode'])

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


In [114]:
# Dropping Postalcode to avoid duplicate
dropped_lat_long=lat_long.drop(columns='Postal Code')
#dropped_lat_long

In [115]:
#merging geospadial data with postalcode dataframe. 
final_df = pd.concat([groupby1,dropped_lat_long], axis=1)

GEOSPATIAL DATAFRAME OF TORONTO IN ALPHABETICAL ORDER IN POSTALCODE

In [115]:
final_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
